In [1]:
!cd ../; sudo python3 -m pip uninstall -y g4ppyy; sudo python3 setup.py install

Found existing installation: g4ppyy 0.1.0
Uninstalling g4ppyy-0.1.0:
  Successfully uninstalled g4ppyy-0.1.0
running install
running bdist_egg
running egg_info
writing g4ppyy.egg-info/PKG-INFO
writing dependency_links to g4ppyy.egg-info/dependency_links.txt
writing requirements to g4ppyy.egg-info/requires.txt
writing top-level names to g4ppyy.egg-info/top_level.txt
file macros.py (for module macros) not found
reading manifest file 'g4ppyy.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'g4ppyy.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
file macros.py (for module macros) not found
file macros.py (for module macros) not found
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/g4ppyy
copying build/lib/g4ppyy/magic.py -> build/bdist.linux-x86_64/egg/g4ppyy
copying build/lib/g4ppyy/_visualisers.py -> build/bdist.linux-x86_64/egg/g4ppyy
copying build/lib/g4ppyy

In [2]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
import os
# os.environ["PYTHONPATH"]="/custom"

import pandas as pd
import matplotlib.pyplot as plt

import g4ppyy as g4
from g4ppyy.SI import mm, cm, m, eV, MeV, GeV, deg, twopi

import random

gRunManager = g4.G4RunManager();

class custom_generator(g4.G4VUserPrimaryGeneratorAction):
    "Moderated Cf252 Generator"
    def __init__(self):
        super().__init__()
        self.particle = g4.G4Neutron.Definition()
        self.particleGun = g4.G4ParticleGun()
        self.particleGun.SetParticleDefinition(self.particle)
        self.particleGun.SetParticleMomentumDirection(g4.G4ThreeVector(1, 0, 0))
        self.particleGun.SetNumberOfParticles(1)  
        self.particleGun.SetParticleTime(0)

    def GeneratePrimaries(self, anEvent):
        energy = 14 * MeV #beam_energy
        self.particleGun.SetParticlePosition(g4.G4ThreeVector(0.0,0.0,0.0))
        self.particleGun.SetParticleEnergy(energy)  
        self.particleGun.GeneratePrimaryVertex(anEvent)


class neutron_tracker(g4.G4VSensitiveDetector):

    def Reset(self):
        self.neutron_event = {
            "eid": [],
            "edep": [],
            "x": [],
            "y": [],
            "z": []        
        }
                
    def ProcessHits(self, aStep, ROhist):
        
        pdg = (aStep.GetTrack().GetParticleDefinition().GetPDGEncoding())
        if not (pdg == 2112): return 0

        eid = int(gRunManager.GetCurrentEvent().GetEventID())

        self.neutron_event["eid"].append( eid )
        
        pos = aStep.GetPreStepPoint().GetPosition() 
        dirs = aStep.GetTrack().GetMomentumDirection() 
        ek = aStep.GetPreStepPoint().GetTotalEnergy() 

        self.neutron_event["edep"].append(aStep.GetTotalEnergyDeposit())
        self.neutron_event["x"].append(pos.x())
        self.neutron_event["y"].append(pos.y())
        self.neutron_event["z"].append(pos.z())
        
        # aStep.GetTrack().SetTrackStatus(g4.G4TrackStatus.fStopAndKill)

        return 1

    def StartOfRunAction(self):
        self.Reset()

    def EndOfRunAction(self):
        self.df = pd.DataFrame(data=self.neutron_event)

    def VisualizationAction(self):
        
        plt.scatter(self.df.x, self.df.y, c=self.df.edep)
        plt.title(str(self.GetName()) + " : XY Edep")
        plt.xlabel("x [mm]")
        plt.ylabel("z [mm]")
        plt.show()
        
        plt.scatter(self.df.x, self.df.z, c=self.df.edep)
        plt.title(str(self.GetName()) + " : XZ Edep")
        plt.xlabel("x [mm]")
        plt.xlabel("z [mm]")
        
        
        plt.show()
     
# In[3]:

class custom_world(g4.G4VUserDetectorConstruction):   
    def BuildMaterials(self):
        # Material definitions
        self.water_mat = g4.gNistManager.FindOrBuildMaterial("G4_WATER")

    def BuildWorld(self):
        # Mother Box
        self.world = g4.builder.build_component("world", solid="box", x=5*m, y=5*m, z=5*m, material=self.water_mat)

        # World Geometries
        self.hdpe_outer = g4.builder.build_component("shell", solid="tubs", rot=[90*deg, 0.0, 0.0], rmax=22*cm, z=1.2*m/2, mother=self.world, material="G4_POLYETHYLENE", color=[0.0,0.0,1.0,0.8], drawstyle="solid")
        self.water_inner = g4.builder.build_component("water", solid="tubs", rmax=18*cm, z=1.2*m/2, mother=self.hdpe_outer, material="G4_WATER", color=[0.5,0.5,1.0,0.1], drawstyle="solid")
        self.hdpe_endcap1 = g4.builder.build_component("cap1", solid="tubs", rmax=18*cm, z=1*cm, mother=self.water_inner, material="G4_POLYETHYLENE", color=[1.0,0.2,1.0,0.8], drawstyle="solid", pos=[0.0, 0.0, +1.2*m/2 - 1*cm])
        self.hdpe_endcap2 = g4.builder.build_component("cap2", solid="tubs", rmax=18*cm, z=1*cm, mother=self.water_inner, material="G4_POLYETHYLENE", color=[1.0,0.2,1.0,0.8], drawstyle="solid", pos=[0.0, 0.0, -1.2*m/2 + 1*cm])

        self.hdpe_endcap3 = g4.builder.build_component("cap3", solid="sphere", rmax=18*cm, z=1*cm, mother=self.world, material="G4_POLYETHYLENE", color=[1.0,0.1,1.0,0.8], drawstyle="solid", pos=[+2.0*m, 0.0, -1.2*m/2 + 1*cm])
        self.hdpe_endcap4 = g4.builder.build_component("cap4", solid="box", x=30*cm, y=12*cm, z=10*cm, mother=self.world, material="G4_POLYETHYLENE", color=[1.0,1.0,0.1,0.8], drawstyle="solid", pos=[0.0*m, +1.0*m, -1.2*m/2 + 1*cm])

    
    def BuildDetectors(self):
        # Sensitive Volume Definitions
        self.hdpe_det = neutron_tracker("hdpe_det")
        self.world_det = neutron_tracker("world_det")
        self.world_det.Reset()
        self.hdpe_det.Reset()
        
        
        self.world.GetLogicalVolume().SetSensitiveDetector(self.world_det)
        self.hdpe_outer.GetLogicalVolume().SetSensitiveDetector(self.hdpe_det)

        g4.run.register_detector_hooks(detector.hdpe_det)
        g4.run.register_detector_hooks(detector.world_det)

    def Construct(self):
        self.BuildMaterials()
        self.BuildWorld()
        self.BuildDetectors()
        
        # Return the mother
        return self.world # top mother volume



[G4PPYY] : Geant4 Python wrapper for CPPYY
[G4PPYY] : Author: P. Stowell (p.stowell@sheffield.ac.uk)
[G4PPYY] :         R. Foster
[G4PPYY] : Loading G4 Modules.
[G4PPYY] : G4PREFIX : /app/geant4-v11.2.2/install
[G4PPYY] : G4VERSION : 11.2.2
[G4PPYY] : Module loading complete.
Jupyter Magic : g4_k3d g4_compile
[G4PPYY] : Imported all definitions.

**************************************************************
 Geant4 version Name: geant4-11-02-patch-02 [MT]   (21-June-2024)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



In [3]:
# In [4]:
# Add Physics List
physics = g4.QGSP_BERT()
gRunManager.SetUserInitialization(physics)

# Add a World
detector = custom_world()
gRunManager.SetUserInitialization(detector)

# Add a Generator
gen = custom_generator()
gRunManager.SetUserAction(gen)

# Add standard GEANT4 Actions
g4.run.add_default_actions(gRunManager)

# Setup vis to check geometry (optional)
# g4.run.create_visualization(gRunManager)

# traj_mc = g4.mc.vis.modeling.trajectories
# traj_mc.create.drawByParticleID()
# traj_mc.drawByParticleID_0.default.setDrawStepPts(True)
# traj_mc.drawByParticleID_0.default.setStepPtsSize(1)
# traj_mc.drawByParticleID_0.set("e+","white")
# traj_mc.drawByParticleID_0.set("e-","white")
# traj_mc.drawByParticleID_0.set("gamma","yellow")
# traj_mc.drawByParticleID_0.set("neutron","magenta")
# traj_mc.drawByParticleID_0.set("proton","blue")
# traj_mc.drawByParticleID_0.set("pi+","red")
# traj_mc.drawByParticleID_0.set("pi-","red")
# traj_mc.drawByParticleID_0.set("pi0","grey")
# traj_mc.drawByParticleID_0.set("mu-","red")
# traj_mc.drawByParticleID_0.set("mu+","red")
# traj_mc.drawByParticleID_0.set("opticalphoton","white")


g4.macros.enable_drawbyparticleid()
g4.macros.open_vis()
g4.macros.draw_vis()

gRunManager.Initialize()
scoringManager = g4.G4ScoringManager.GetScoringManager()


print("FINISHED")

<<< Geant4 Physics List simulation engine: QGSP_BERT


 hInelastic QGSP_BERT  Thresholds: 
    1) between BERT  and FTF/P over the interval 3 to 6 GeV. 
    2) between FTF/P and QGS/P over the interval 12 to 25 GeV. 
  -- quasiElastic: 1 for QGS  and 0 for FTF
### Adding tracking cuts for neutron  TimeCut(ns)= 10000  KinEnergyCut(MeV)= 0
FINISHED


In [ ]:
%%g4_macro
/score/create/boxMesh neutronDetector
/score/mesh/boxSize 100 100 100 cm
/score/mesh/nBin 4 4 4
/score/mesh/translate/xyz 0 0 0 cm

# Define the flux scorer
/score/quantity/flatSurfaceCurrent neutronFlux
/score/filter/particle neutronFilter neutron
/score/close

# Start the run
/run/beamOn 10000

# Output the results to a file
/score/dumpQuantityToFile neutronDetector neutronFlux neutronFlux.out

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("neutronFlux.out", skiprows=2)
print(df.columns)
plt.hist2d(df["# iX"], df[" iY"], weights=df[" total(value) [percm2]"], bins=[np.unique(df["# iX"]),np.unique(df["# iX"])])
plt.show()